In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.3)

### Линейная регрессия
Перед началом работу догрузим необходимые нам функции



In [3]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

Рассмотрим данные исследования 2004 года о зависимости риска развития диабета от пола, возраста, индекса массы тела, среднего кровяного давления и других показателей. 



#### Загрузка данных

In [4]:
data = load_diabetes()
data

{'data': array([[ 0.03807591,  0.05068012,  0.06169621, ..., -0.00259226,
          0.01990749, -0.01764613],
        [-0.00188202, -0.04464164, -0.05147406, ..., -0.03949338,
         -0.06833155, -0.09220405],
        [ 0.08529891,  0.05068012,  0.04445121, ..., -0.00259226,
          0.00286131, -0.02593034],
        ...,
        [ 0.04170844,  0.05068012, -0.01590626, ..., -0.01107952,
         -0.04688253,  0.01549073],
        [-0.04547248, -0.04464164,  0.03906215, ...,  0.02655962,
          0.04452873, -0.02593034],
        [-0.04547248, -0.04464164, -0.0730303 , ..., -0.03949338,
         -0.00422151,  0.00306441]]),
 'target': array([151.,  75., 141., 206., 135.,  97., 138.,  63., 110., 310., 101.,
         69., 179., 185., 118., 171., 166., 144.,  97., 168.,  68.,  49.,
         68., 245., 184., 202., 137.,  85., 131., 283., 129.,  59., 341.,
         87.,  65., 102., 265., 276., 252.,  90., 100.,  55.,  61.,  92.,
        259.,  53., 190., 142.,  75., 142., 155., 225.,  59

Функция `sklearn.datasets.load_diabetes()` возвращает словарь. В поле `data` записана матрица регрессоров, в которой данные предварительно центрированы и нормированы. В поле `target` записана мера прогрессирования заболевания в течении года. В поле `DESCR` можно прочитать подробнее о данных.

Посмотрим на описание датасета.

In [5]:
print(data['DESCR'])

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

Note: Each of these 1

Поле `data` содержит матрицу размера 442 $\times$ 10, где 442 — количество пациентов, а 10 — количество признаков (возраст, пол, и т.д.). 
Строки матрицы соответствуют пациентам, столбцы — признакам.

In [6]:
data['data'].shape

(442, 10)

Целевая переменная $-$ мера прогрессирования заболевания в течении года.

In [7]:
data['target'].shape

(442,)

Создайте матрицу регрессоров $X$ и столбец наблюдений $y$.

In [9]:
X, y = data['data'], data['target']

#### Обучение моделей

Разбейте данные случайно на две части — обучающую и тестовую в соотношении 80:20.

Если что-то забыли или что-то не понятно, можно почитать справку:

In [8]:
train_test_split?

Signature:
train_test_split(
    *arrays,
    test_size=None,
    train_size=None,
    random_state=None,
    shuffle=True,
    stratify=None,
)
Docstring:
Split arrays or matrices into random train and test subsets.

Quick utility that wraps input validation,
``next(ShuffleSplit().split(X, y))``, and application to input data
into a single call for splitting (and optionally subsampling) data into a
one-liner.

Read more in the :ref:`User Guide <cross_validation>`.

Parameters
----------
*arrays : sequence of indexables with same length / shape[0]
    Allowed inputs are lists, numpy arrays, scipy-sparse
    matrices or pandas dataframes.

test_size : float or int, default=None
    If float, should be between 0.0 and 1.0 and represent the proportion
    of the dataset to include in the test split. If int, represents the
    absolute number of test samples. If None, the value is set to the
    complement of the train size. If ``train_size`` is also None, it will
    be set to 0.25.

trai

При разбиении датасета стоит зафиксировать случайность для воспроизводимости результатов, поставив `random_state=42` в функцию разбиения (можете ставить любое число, необязательно именно 42)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(331, 10) (111, 10) (331,) (111,)


Заведите модель линейной регрессии из `sklearn` и обучите ее по обучающей части данных.

In [12]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

Выведите кооэффициенты модели, включая свободный член.

In [13]:
model.coef_

array([  47.74968054, -241.99090728,  531.97106288,  381.56286182,
       -918.50290455,  508.25778252,  116.95016447,  269.4923028 ,
        695.80811712,   26.32458203])

Посчитайте предсказания построенной модели на тестовой выборке

In [14]:
y_pred = model.predict(X_test)

Реализуйте метрики MSE, MAE, MAPE без использования `sklearn` и других готовых реализаций.

In [27]:
import scipy.stats as sps

def mean_squared_error(y_true, y_pred):
    return np.mean((y_pred-y_true)**2),sps.sem((y_pred-y_true)**2)

def mean_absolute_error(y_true, y_pred):
    return np.mean(np.abs(y_pred-y_true)),sps.sem(np.abs(y_pred-y_true))

def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs(y_pred-y_true)/y_true),sps.sem(np.abs(y_pred-y_true)/y_true)

Посчитайте MSE, MAE, MAPE на тестовой выборке и выведите с точностью до трех знаков после запятой.

In [28]:
funcs = [mean_squared_error, mean_absolute_error, mean_absolute_percentage_error,]

names = ['MSE','MAE','MAPE']

for func,name in zip(funcs,names):
    print(name,func(y_test,y_pred))

MSE (2690.9435850075597, 301.3858147463307)
MAE (43.34579112124662, 2.717093997572367)
MAPE (0.38674679783870475, 0.04066295555556105)


### Различные разбиения

Разбейте данные на тренировочную и тестовую выборки еще раз: измените пераметр `random_state`. Для каждого из разбиений обучите модель и получите метрики качества. Меняются ли эти метрики, если менять разбиение данных?

In [29]:
mse = []
mae = []
mape = []
states = [1, 495, 324, 8, 0, 900]
for state in states:

    # разбиение данных
    X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=state)
    
    # обучение модели
    model.fit(X_train,y_train)

    # предсказание модели
    y_pred = model.predict(X_test)

    # подсчет метрики на тестовой выборке
    mse.append(mean_squared_error(y_test,y_pred))
    mae.append(mean_absolute_error(y_test,y_pred))
    mape.append(mean_absolute_percentage_error(y_test,y_pred))

In [38]:
np.array([mse, mae, mape]).shape

(3, 6, 2)

In [52]:
print('Значение метрик')
display(pd.DataFrame(np.array([mse, mae, mape])[:,:,0].T,index=states,columns=['MSE','MAE','MAPE']))
print('Выборочное стандратное отклонение')
display(pd.DataFrame(np.array([mse, mae, mape])[:,:,1].T,index=states,columns=['MSE','MAE','MAPE']))

Значение метрик


,MSE,MAE,MAPE
1,2903.126734,41.982981,0.328536
495,2782.889666,42.133147,0.385586
324,2945.730623,44.308199,0.417141
8,3108.051631,45.239111,0.408759
0,3180.159648,45.120563,0.379610
900,2690.943585,43.345791,0.386747


Выборочное стандратное отклонение


,MSE,MAE,MAPE
1,418.277465,3.220045,0.031623
495,401.426535,3.026681,0.043886
324,360.941714,2.988636,0.038926
8,408.957624,3.106407,0.048485
0,438.406370,3.225318,0.041339
900,301.385815,2.717094,0.040663


**Вывод:** предсказания различаются при смене зерна, но не стат значимо. Опровергнем гипотезу о различие метрик на уровне $\alpha = 0.05$